# PROJETO ANÁLISE PREDITIVA - CONSUMO ENERGIA ELÉTRICA

O objetivo dessa análise preditiva é identificar os grupos de consumidores com a técnica de aprendizagem não supervisionada, ou seja, identificar os clusters presentes no dataset. Este é um projeto prático feito ao longo do curso Data Science Academy e que foi aperfeiçoado antes da publicação no GitHub. 

Temos um dataset com o consumo de energia elétrica dos últimos 4 anos no nível doméstico. 
A fonte dos dados é: https://archive.ics.uci.edu/ml/datasets/Individual+household+electric+power+consumption

## 1 - Carga de Pacotes

Nesta seção será carregados todos os pacotes necessários para a execução do script em Python.

In [1]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import pylab
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from scipy.spatial.distance import cdist, pdist
from sklearn.metrics import silhouette_score
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

## 2 - Carga dos Dados

Nesta seção será feita a carga dos dados disponibilizados pelo UCI.

In [5]:
# Carregando os dados
bd_power_consumption = pd.read_csv('dados/household_power_consumption.txt', delimiter = ';', low_memory = False)
bd_power_consumption.head()

,Date,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
0,16/12/2006,17:24:00,4.216,0.418,234.840,18.400,0.000,1.000,17.0
1,16/12/2006,17:25:00,5.360,0.436,233.630,23.000,0.000,1.000,16.0
2,16/12/2006,17:26:00,5.374,0.498,233.290,23.000,0.000,2.000,17.0
3,16/12/2006,17:27:00,5.388,0.502,233.740,23.000,0.000,1.000,17.0
4,16/12/2006,17:28:00,3.666,0.528,235.680,15.800,0.000,1.000,17.0


## 3 - Análise Explorátoria

Seção destinada a uma breve análise exploratória dos dados para um melhor entedimento relacionado a relação das variáveis.

In [7]:
# Verifica os tipos de dados
# Dados estão como Object, como será feito o uso do PCA não é necessário alterara o tipo ao menos nesse momento da análise
bd_power_consumption.dtypes

Date                      object
Time                      object
Global_active_power       object
Global_reactive_power     object
Voltage                   object
Global_intensity          object
Sub_metering_1            object
Sub_metering_2            object
Sub_metering_3           float64
dtype: object

In [9]:
# Verifica o shape da base
# Confirma que todos os dados foram carredados
bd_power_consumption.shape

(2075259, 9)

In [ ]:
# Verifica se há valores nulos
dataset.isnull().values.any()

# 4 - Pré-Processamento dos Dados

Seção destinada ao Pré-Processamento dos dados, como uso de PCA e normalização dos dados.

# 5 - Modelo Preditivo

Aplicação do modelo preditivo para identificação dos clusters

# 6 - Análise dos Resultados